<a href="https://colab.research.google.com/github/AjayKumarGogineni777/Heart-Sound-Segmentation/blob/master/U_Net_mod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import scipy.io as sio

#mat_contents = sio.loadmat('/content/gdrive/My Drive/1D U Net/features_normal.mat')
#labels = sio.loadmat('/content/gdrive/My Drive/1D U Net/states_normal.mat')

mat_contents = sio.loadmat('/content/gdrive/My Drive/1D U Net/LSTM_classification/LS1_features_all_no_EWT')
labels = sio.loadmat('/content/gdrive/My Drive/1D U Net/states_normal.mat')

In [0]:
k = 770
t1 = mat_contents['LS1_features'][0][k]
t1_shape = mat_contents['LS1_features'][0][k].shape
t1_shape

(100, 4)

In [0]:
cnt3 = 0
for i in range(792):
  a = labels['LS1_states'][0][i].shape
  cnt3 = cnt3+a[0]

In [0]:
cnt3/8

62340.625

In [0]:
1475//64

23

In [0]:
import numpy as np
import torch
elements = 56339
temp2 = torch.randn(elements,64)
cnt2 = 0
for i in range(792):
  a = labels['concat_states'][0][i].shape
  num = a[0]//8
  #print(num)
  for j in range(num):
    if j*8+64< a[0]:
      a2 = labels['concat_states'][0][i][j*8:(j*8+64)]
    
      a2 = torch.from_numpy(np.transpose(a2))
      temp2[cnt2] = a2
      cnt2 = cnt2+1
      
temp = torch.randn(elements, 4, 64)
cnt2 = 0
for i in range(792):
  a = mat_contents['LS1_features'][0][i].shape
  num = a[0]//8   
  
  for j in range(num):
    if j*8+64< a[0]:
      a2 = mat_contents['LS1_features'][0][i][j*8:(j*8+64)]
    
      a2 = torch.from_numpy(np.transpose(a2))
      temp[cnt2] = a2
      cnt2 = cnt2+1

In [0]:
temp2[50000]

tensor([1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3.,
        4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 1., 1.,
        1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3.,
        4., 4., 4., 4., 4., 4., 4., 4., 4., 4.])

In [0]:
#### Main code
## Using blocks

import torch
import torch.nn as nn
import torch.nn.functional as F


class double_conv(nn.Module):
    '''(conv => BN => ReLU ) * 1'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(in_ch, out_ch, 5,1, 2),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv1d(out_ch, out_ch, 5, 1, 2),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        x = self.conv(x)
        return x
      
class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x   

class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool1d(2),
            double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x      

      
class up(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(up, self).__init__()

        self.up = nn.ConvTranspose1d(in_ch, in_ch//2,4,2,1)

        self.conv = double_conv(in_ch, out_ch)
    
    def forward(self, x1, x2):
        ### x2 is from downsampling path
        x1 = self.up(x1)
        
        # input is CHW
        #diffY = x2.size()[2] - x1.size()[2]
        #diffX = x2.size()[3] - x1.size()[3]

        #x1 = F.pad(x1, (diffX // 2, diffX - diffX//2,
        #                diffY // 2, diffY - diffY//2))
        
        # for padding issues, see 
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd

        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x   
class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv1d(in_ch, out_ch,5, 1, 2)

    def forward(self, x):
        x = self.conv(x)
        return x      
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        #n_channels = 4, n_classes = 4
        self.inc = inconv(4, 8)
        self.down1 = down(8, 16)
        self.down2 = down(16, 32)
        self.down3 = down(32, 64)
        self.down4 = down(64, 128)
        
        self.up1 = up(128, 64)
        self.up2 = up(64, 32)
        self.up3 = up(32, 16)
        self.up4 = up(16, 8)
        
        
        self.outc = outconv(8, 4)
    
    def forward(self, x):
      
        x1 = self.inc(x)     
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        #return F.sigmoid(x)
        return x
model = UNet()
sample = temp[64:128]
target = temp2[0]
out = model(sample)
out.size()      




### ADAM optimiser

### training loop


def softmax_mod(x): return (x.exp()/(x.exp().sum(-1,keepdim=True))).log()
def nll_indexing(soft,target):
  ## soft : 64,64,4
  ## target : 64,64,4 or 64, 64
  
  return -soft[range(target.shape[0]),range(target.shape[1]), target-1].mean()

def nll_final(soft,target):
  batch_var = torch.zeros([64, 64] , requires_grad=True)
  out = torch.zeros([1, 1] , requires_grad=True)
  for i in range(64):
    for j in range(64):
      ind = target[i][j]-1
      batch_var[i][j] = soft[i][j][ind]
      out = out + soft[i][j][ind]
  #out = batch_var.mean()
  return -1*(out/(64*64))


#soft = softmax_mod(out).transpose(1,2)
#soft.shape
lr = 1e-4
epochs = 4
#n = 7391
bs = 64
loss_func = F.cross_entropy
loss_func2 = torch.nn.L1Loss
#def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()

def accuracy_mod(out, yb): return (torch.argmax(out, dim=2)==yb-1).float().mean()
def log_softmax(x): return (x.exp()/(x.exp().sum(-1,keepdim=True))).log()


opt = torch.optim.Adam(model.parameters(), lr=lr)

#model.load_state_dict(torch.load('/content/gdrive/My Drive/1D U Net/EWT/models/EWT_LS1_155_epochs'))
model.load_state_dict(torch.load('/content/gdrive/My Drive/1D U Net/EWT/stride_8/models/speech_8_strides_44_epochs_stride_64'))
for epoch in range(epochs):
    for i in range(100): ### i = 1:100 for training
    #for i in range((n-1)//bs + 1):
#         set_trace()
        start_i = i*bs
        end_i = start_i+bs
        xb = temp[start_i:end_i]
        #yb = y_f[start_i:end_i]
        target = temp2[start_i:end_i]
            
        out = model(xb)
        #loss = loss_func(model(xb), yb.long())
        #loss = nll2((log_softmax(model(xb))),y_onehot.long())
        loss = nll_final((softmax_mod(out.transpose(1, 2))),target.long()) ## Negative log likelihood of log softmax
        #rint(loss)
        #loss = cross_entropy(out.squeeze(),yb)
        #loss = criterion(out.transpose(1, 2), torch.max(yb.transpose(1, 2), 1)[1])
        
        
        accuracy = accuracy_mod(softmax_mod(out.transpose(1, 2)),target.long())

        loss.backward()
      #for param in model.parameters():
        #print(param.grad.data.sum())
        opt.step()
        opt.zero_grad()
    print('accuracy',accuracy)
    print(' > Epoch {:2d} loss: {:.3f}'.format((epoch+1), loss.item()))

accuracy tensor(0.9336)
 > Epoch  1 loss: 0.178
accuracy tensor(0.9348)
 > Epoch  2 loss: 0.178
accuracy tensor(0.9343)
 > Epoch  3 loss: 0.168
accuracy tensor(0.9382)
 > Epoch  4 loss: 0.156


In [0]:
st = model.state_dict()
mdl_path = '/content/gdrive/My Drive/1D U Net/EWT/stride_8/models'
torch.save(st, mdl_path+'/speech_8_strides_48_epochs_stride_64')

In [0]:
cnt_acc = 0
acc_total = 0
#for i in range(650,795,1):
for i in range(90, 105, 1):
  start_i = i*bs
  end_i = start_i+bs
  xb = temp[start_i:end_i]
  #yb = y_f[start_i:end_i]
  target = temp2[start_i:end_i]
            
  out = model(xb)
  #loss = loss_func(model(xb), yb.long())
        #loss = nll2((log_softmax(model(xb))),y_onehot.long())
  loss = nll_final((softmax_mod(out.transpose(1, 2))),target.long()) ## Negative log likelihood of log softmax
  #loss = cross_entropy(out.squeeze(),yb)
  #loss = criterion(out.transpose(1, 2), torch.max(yb.transpose(1, 2), 1)[1])
  accuracy = accuracy_mod(softmax_mod(out.transpose(1, 2)),target.long())
  
  
  acc_total = acc_total + accuracy
  cnt_acc = cnt_acc+1
  #print('loss',loss)
  #print('acc',accuracy)
print(acc_total/cnt_acc)  

tensor(0.9058)


In [0]:
sm_preds = []
bs = 64
#for i in range(750, 790):
for i in range(80, 105, 1):
    
    start_i = i*bs
    end_i = start_i+bs
    xb = temp[start_i:end_i]
    
    target = temp2[start_i:end_i]
        
    out = model(xb)
    sm_preds_val = softmax_mod(out.transpose(1, 2))
    sm_preds_fn = torch.argmax(sm_preds_val, dim=2)+1
    sm_preds.append(sm_preds_fn)

In [0]:

target_comp = []
#for i in range(750, 790):
for i in range(80, 105, 1):
    start_i = i*bs
    end_i = start_i+bs
    xb = temp[start_i:end_i]
    #yb = y_f[start_i:end_i]
    target = temp2[start_i:end_i]
    target_comp.append(target)
    

In [0]:
tr = []
pred = []
for i in range(25):
  s1_true = np.where(target_comp[i][0] == 1)
  s1_pred = np.where(sm_preds[i][0] == 1)
  tr.append(s1_true)
  pred.append(s1_pred)

In [0]:
for i in range(25):
    print(i)
    print(tr[i])
    print(pred[i])

0
(array([ 1,  2,  3,  4,  5,  6,  7, 44, 45, 46, 47, 48, 49, 50]),)
(array([ 1,  2,  3,  4,  5,  6,  7,  8, 44, 45, 46, 47, 48, 49]),)
1
(array([32, 33, 34, 35, 36, 37, 38]),)
(array([32, 33, 34, 35, 36, 37, 38]),)
2
(array([10, 11, 12, 13, 14, 15, 16, 49, 50, 51, 52, 53, 54, 55]),)
(array([ 8,  9, 10, 11, 12, 13, 14, 49, 50, 51, 52, 53, 54, 55]),)
3
(array([ 4,  5,  6,  7,  8,  9, 10, 50, 51, 52, 53, 54, 55, 56]),)
(array([ 5,  6,  7,  8,  9, 10, 11, 50, 51, 52, 53, 54, 55, 56]),)
4
(array([49, 50, 51, 52, 53, 54, 55]),)
(array([49, 50, 51, 52, 53, 54, 55]),)
5
(array([24, 25, 26, 27, 28, 29, 30]),)
(array([23, 24, 25, 26, 27, 28, 29]),)
6
(array([ 0, 41, 42, 43, 44, 45, 46, 47]),)
(array([ 0,  1,  2,  3, 40, 41, 42, 43, 44, 45, 46]),)
7
(array([ 0, 36, 37, 38, 39, 40, 41, 42]),)
(array([ 0,  1,  2, 39, 40, 41, 42, 43, 44, 45]),)
8
(array([ 6,  7,  8,  9, 10, 11, 12, 45, 46, 47, 48, 49, 50, 51]),)
(array([ 7,  8,  9, 10, 11, 12, 13, 46, 47, 48, 49, 50, 51]),)
9
(array([ 5,  6,  7,  8

In [0]:
tr = []
pred = []
for i in range(25):
  s1_true = np.where(target_comp[i][0] == 3)
  s1_pred = np.where(sm_preds[i][0] == 3)
  tr.append(s1_true)
  pred.append(s1_pred)
for i in range(25):
    print(i)
    print(tr[i])
    print(pred[i])  

0
(array([20, 21, 22, 23, 24, 62, 63]),)
(array([20, 21, 22, 23, 62, 63]),)
1
(array([ 9, 10, 11, 12, 13, 47, 48, 49, 50, 51]),)
(array([ 9, 10, 11, 12, 13, 47, 48, 49, 50, 51]),)
2
(array([28, 29, 30, 31, 32, 62, 63]),)
(array([30, 31, 32, 33, 62, 63]),)
3
(array([24, 25, 26, 27, 28]),)
(array([23, 24, 25, 26, 27]),)
4
(array([ 8,  9, 10, 11, 12]),)
(array([ 8,  9, 10, 11, 12]),)
5
(array([39, 40, 41, 42, 43]),)
(array([40, 41, 42, 43]),)
6
(array([14, 15, 16, 17, 18, 60, 61, 62, 63]),)
(array([14, 15, 16, 17, 18, 59, 60, 61, 62]),)
7
(array([13, 14, 15, 16, 17, 54, 55, 56, 57, 58]),)
(array([13, 14, 15, 16, 17, 55, 56, 57, 58, 59]),)
8
(array([23, 24, 25, 26, 27, 62, 63]),)
(array([23, 24, 25, 26, 27]),)
9
(array([20, 21, 22, 23, 24, 57, 58, 59, 60, 61]),)
(array([20, 21, 22, 23, 24, 57, 58, 59, 60]),)
10
(array([33, 34, 35, 36, 37]),)
(array([34, 35, 36, 37, 38]),)
11
(array([ 7,  8,  9, 10, 11, 41, 42, 43, 44, 45]),)
(array([ 7,  8,  9, 10, 11, 40, 41, 42, 43, 44]),)
12
(array([10,

In [0]:
target[18]

tensor([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4., 4., 4., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2.,
        2., 3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4., 4., 4., 4., 4., 4., 4.])

In [0]:
sm_preds[0][18]

tensor([2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
        2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [0]:
k = 18
sm_final = sm_preds[0][k]
for j in range(1,63):
  #print(j) 1 to 62
  if sm_preds[0][k][j] == (sm_final[j-1]+1)//4:
    sm_final[j] = sm_preds[0][k][j]
  else:
    sm_final[j] = sm_final[j-1]

    
sm_final  

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4])

In [0]:
sm_final

In [0]:
56339//64

880

In [0]:
i = 880
bs = 64
start_i = i*bs
end_i = start_i+bs
xb = temp[start_i:end_i]

In [0]:
xb

In [0]:
temp[0].shape

torch.Size([4, 64])

In [0]:

import numpy as np
import torch
elements = 7104
temp2 = torch.randn(elements, 64)
cnt2 = 0
for i in range(760):
  a = labels['LS1_states'][0][i].shape
  num = a[0]//64
  #print(num)
  for j in range(num):
    a2 = labels['LS1_states'][0][i][j*64:(j*64+64)]
    a2 = torch.from_numpy(np.asarray(a2))
    temp2[cnt2] = a2[0]
    cnt2 = cnt2+1
    #a2 = torch.from_numpy(np.transpose(a2))

In [0]:
temp2.shape

torch.Size([7104, 64])

In [0]:
## Satets/ Labels

import numpy as np
import torch
elements = 7104
temp2 = torch.randn(elements,64)
cnt2 = 0
for i in range(760):
  a = labels['LS1_states'][0][i].shape
  num = a[0]//64
  #print(num)
  for j in range(num):
    a2 = labels['LS1_states'][0][i][j*64:(j*64+64)]
    
    a2 = torch.from_numpy(np.transpose(a2))
    temp2[cnt2] = a2
    cnt2 = cnt2+1

In [0]:
y_f = torch.zeros(elements,4,64)
for i in range(elements):
  for j in range(4):
      for k in range(64):
        if temp2[i][k] == 1:
          y_f[i][0][k] = 1
        elif temp2[i][k] == 2:
          y_f[i][1][k] = 1
        elif temp2[i][k] == 3:
          y_f[i][2][k] = 1
        elif temp2[i][k] == 4:
          y_f[i][3][k] = 1
            

In [0]:
y_f[0][0][0:63]

tensor([0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
        1., 1., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
115*64

7360

In [0]:
## Features
import numpy as np
import torch
elements = 7104
temp = torch.randn(elements,4,64)
cnt = 0
for i in range(760):
  a = mat_contents['LS1_features'][0][i].shape
  num = a[0]//64
  #print(num)
  for j in range(num):
    a2 = mat_contents['LS1_features'][0][i][j*64:(j*64+64)]
    a2 = torch.from_numpy(np.transpose(a2))
    #a2 = torch.from_numpy(np.array(a2))
    temp[cnt] = a2
    cnt = cnt+1
  

In [0]:
## Using blocks

import torch
import torch.nn as nn
import torch.nn.functional as F


class double_conv(nn.Module):
    '''(conv => BN => ReLU ) * 1'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(in_ch, out_ch, 5,1, 2),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv1d(out_ch, out_ch, 5, 1, 2),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        x = self.conv(x)
        return x
      
class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x   

class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool1d(2),
            double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x      

      
class up(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(up, self).__init__()

        self.up = nn.ConvTranspose1d(in_ch, in_ch//2,4,2,1)

        self.conv = double_conv(in_ch, out_ch)
    
    def forward(self, x1, x2):
        ### x2 is from downsampling path
        x1 = self.up(x1)
        
        # input is CHW
        #diffY = x2.size()[2] - x1.size()[2]
        #diffX = x2.size()[3] - x1.size()[3]

        #x1 = F.pad(x1, (diffX // 2, diffX - diffX//2,
        #                diffY // 2, diffY - diffY//2))
        
        # for padding issues, see 
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd

        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x   
class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv1d(in_ch, out_ch,5, 1, 2)

    def forward(self, x):
        x = self.conv(x)
        return x      

In [0]:
#### Main code

import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        #n_channels = 4, n_classes = 4
        self.inc = inconv(4, 8)
        self.down1 = down(8, 16)
        self.down2 = down(16, 32)
        self.down3 = down(32, 64)
        self.down4 = down(64, 128)
        
        self.up1 = up(128, 64)
        self.up2 = up(64, 32)
        self.up3 = up(32, 16)
        self.up4 = up(16, 8)
        
        
        self.outc = outconv(8, 4)
    
    def forward(self, x):
      
        x1 = self.inc(x)     
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        #return F.sigmoid(x)
        return x
model = UNet()
sample = temp[64:128]
target = temp2[0]
out = model(sample)
out.size()      

In [0]:
model = UNet()
sample = temp[64:128]
target = temp2[0]
out = model(sample)
out.size()

torch.Size([64, 4, 64])

In [0]:
### training loop


def softmax_mod(x): return (x.exp()/(x.exp().sum(-1,keepdim=True))).log()
def nll_indexing(soft,target):
  ## soft : 64,64,4
  ## target : 64,64,4 or 64, 64
  
  return -soft[range(target.shape[0]),range(target.shape[1]), target-1].mean()

def nll_final(soft,target):
  batch_var = torch.zeros([64, 64] , requires_grad=True)
  out = torch.zeros([1, 1] , requires_grad=True)
  for i in range(64):
    for j in range(64):
      ind = target[i][j]-1
      batch_var[i][j] = soft[i][j][ind]
      out = out + soft[i][j][ind]
  #out = batch_var.mean()
  return -1*(out/(64*64))


#soft = softmax_mod(out).transpose(1,2)
#soft.shape
lr = 5e-4
epochs = 10
n = 7391
bs = 64
loss_func = F.cross_entropy
loss_func2 = torch.nn.L1Loss
#def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()

def accuracy_mod(out, yb): return (torch.argmax(out, dim=2)==yb-1).float().mean()
def log_softmax(x): return (x.exp()/(x.exp().sum(-1,keepdim=True))).log()

for epoch in range(epochs):
    for i in range(100): ### i = 1:100 for training
    #for i in range((n-1)//bs + 1):
#         set_trace()
        start_i = i*bs
        end_i = start_i+bs
        xb = temp[start_i:end_i]
        yb = y_f[start_i:end_i]
        target = temp2[start_i:end_i]
            
        out = model(xb)
        #loss = loss_func(model(xb), yb.long())
        #loss = nll2((log_softmax(model(xb))),y_onehot.long())
        loss = nll_final((softmax_mod(out.transpose(1, 2))),target.long()) ## Negative log likelihood of log softmax
        #rint(loss)
        #loss = cross_entropy(out.squeeze(),yb)
        #loss = criterion(out.transpose(1, 2), torch.max(yb.transpose(1, 2), 1)[1])
        
        
        accuracy = accuracy_mod(softmax_mod(out.transpose(1, 2)),target.long())
        #print('accuracy',accuracy)
        
        #print('accuracy:',accuracy1(model(xb), yb.long()))
        loss.backward()
        #print(out[0])
        #print(yb[0])
        with torch.no_grad():
            for p in model.parameters(): p -= p.grad * lr
            model.zero_grad()
    print(' > Epoch {:2d} loss: {:.3f}'.format((epoch+1), loss.item()))
    print('accuracy',accuracy)

 > Epoch  1 loss: 1.302
accuracy tensor(0.5154)
 > Epoch  2 loss: 1.293
accuracy tensor(0.5193)
 > Epoch  3 loss: 1.283
accuracy tensor(0.5190)
 > Epoch  4 loss: 1.273
accuracy tensor(0.5208)
 > Epoch  5 loss: 1.263
accuracy tensor(0.5193)
 > Epoch  6 loss: 1.253
accuracy tensor(0.5178)
 > Epoch  7 loss: 1.243
accuracy tensor(0.5195)
 > Epoch  8 loss: 1.233
accuracy tensor(0.5190)
 > Epoch  9 loss: 1.222
accuracy tensor(0.5203)
 > Epoch 10 loss: 1.212
accuracy tensor(0.5210)


In [0]:
PATH = '/content/gdrive/My Drive/1D U Net/models/16_epochs'
torch.save(model, PATH)
torch.max(softmax_mod(model(temp[0:64])[0]).transpose(0,1),1)[1]+1

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type UNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type inconv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type double_conv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type down. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-p

tensor([1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 3, 3, 1, 1,
        3, 1, 1, 3, 2, 2, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 2, 3, 4, 4, 4, 4,
        4, 4, 2, 3, 1, 1, 3, 1, 1, 1, 3, 3, 2, 4, 1, 1])

In [0]:
PATH = '/content/gdrive/My Drive/1D U Net/models/16_epochs'
torch.save(model, PATH)
torch.max(softmax_mod(model(temp[0:64])[0]).transpose(0,1),1)[1]+1

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type UNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type inconv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type double_conv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type down. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-p

tensor([1, 3, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 1, 3, 4, 3, 1, 1,
        3, 1, 3, 3, 2, 2, 4, 2, 4, 1, 2, 4, 1, 4, 4, 4, 2, 4, 2, 3, 4, 4, 4, 4,
        4, 4, 2, 3, 4, 1, 3, 1, 1, 1, 3, 3, 1, 4, 1, 1])

In [0]:
torch.max(y_f[0:64][0].transpose(0,1),1)[1]+1

In [0]:

def softmax_mod(x): return (x.exp()/(x.exp().sum(-1,keepdim=True))).log()
def nll_indexing(soft,target):
  ## soft : 64,64,4
  ## target : 64,64,4 or 64, 64
  
  return -soft[range(target.shape[0]),range(target.shape[1]), target-1].mean()

def nll_final(soft,target):
  batch_var = torch.zeros([64, 64] , requires_grad=True)
  out = torch.zeros([1, 1] , requires_grad=True)
  for i in range(64):
    for j in range(64):
      ind = target[i][j]-1
      batch_var[i][j] = soft[i][j][ind]
      out = out + soft[i][j][ind]
  #out = batch_var.mean()
  return -1*(out/(64*64))

In [0]:
### ADAM optimiser

### training loop


def softmax_mod(x): return (x.exp()/(x.exp().sum(-1,keepdim=True))).log()
def nll_indexing(soft,target):
  ## soft : 64,64,4
  ## target : 64,64,4 or 64, 64
  
  return -soft[range(target.shape[0]),range(target.shape[1]), target-1].mean()

def nll_final(soft,target):
  batch_var = torch.zeros([64, 64] , requires_grad=True)
  out = torch.zeros([1, 1] , requires_grad=True)
  for i in range(64):
    for j in range(64):
      ind = target[i][j]-1
      batch_var[i][j] = soft[i][j][ind]
      out = out + soft[i][j][ind]
  #out = batch_var.mean()
  return -1*(out/(64*64))


#soft = softmax_mod(out).transpose(1,2)
#soft.shape
lr = 4e-4
epochs = 4
#n = 7391
bs = 64
loss_func = F.cross_entropy
loss_func2 = torch.nn.L1Loss
#def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()

def accuracy_mod(out, yb): return (torch.argmax(out, dim=2)==yb-1).float().mean()
def log_softmax(x): return (x.exp()/(x.exp().sum(-1,keepdim=True))).log()


opt = torch.optim.Adam(model.parameters(), lr=lr)

#model.load_state_dict(torch.load('/content/gdrive/My Drive/1D U Net/EWT/models/EWT_LS1_155_epochs'))
model.load_state_dict(torch.load('/content/gdrive/My Drive/1D U Net/EWT/models/HIGH_LR_EWT_LS1_23_epochs'))
for epoch in range(epochs):
    for i in range(300): ### i = 1:100 for training
    #for i in range((n-1)//bs + 1):
#         set_trace()
        start_i = i*bs
        end_i = start_i+bs
        xb = temp[start_i:end_i]
        #yb = y_f[start_i:end_i]
        target = temp2[start_i:end_i]
            
        out = model(xb)
        #loss = loss_func(model(xb), yb.long())
        #loss = nll2((log_softmax(model(xb))),y_onehot.long())
        loss = nll_final((softmax_mod(out.transpose(1, 2))),target.long()) ## Negative log likelihood of log softmax
        #rint(loss)
        #loss = cross_entropy(out.squeeze(),yb)
        #loss = criterion(out.transpose(1, 2), torch.max(yb.transpose(1, 2), 1)[1])
        
        
        accuracy = accuracy_mod(softmax_mod(out.transpose(1, 2)),target.long())

        loss.backward()
      #for param in model.parameters():
        #print(param.grad.data.sum())
        opt.step()
        opt.zero_grad()
    print('accuracy',accuracy)
    print(' > Epoch {:2d} loss: {:.3f}'.format((epoch+1), loss.item()))

NameError: ignored

In [0]:
0.178 92.87
0.141 94.8 -> 85 epochs

In [0]:
st = model.state_dict()
mdl_path = '/content/gdrive/My Drive/1D U Net/EWT/models'
torch.save(st, mdl_path+'/HIGH_LR_EWT_LS1_40_epochs')

In [0]:
st = model.state_dict()
mdl_path = '/content/gdrive/My Drive/1D U Net/EWT/models'
torch.save(st, mdl_path+'/EWT_LS1_155_epochs')

In [0]:
PATH = '/content/gdrive/My Drive/1D U Net/models/15_epochs_ADAM'
model = torch.load(PATH)

In [0]:
## Validation
model.eval()
cnt2 = 0
total_acc = 0
val_ids = list(range(80,110,1))
for i in val_ids: ### i = 1:100 for training, 100-114 for validation
    
    #for i in range((n-1)//bs + 1):
#         set_trace()
    start_i = i*bs
    end_i = start_i+bs
    xb = temp[start_i:end_i]
    yb = y_f[start_i:end_i]
    target = temp2[start_i:end_i]
    
    hidden=None
    out = model(xb)
    
    loss = nll_final((softmax_mod(out.transpose(1, 2))),target.long()) ## Negative log likelihood of log softmax
        
    accuracy = accuracy_mod(softmax_mod(out.transpose(1, 2)),target.long())
    total_acc = total_acc + accuracy
    cnt2 = cnt2+1    
    
    print(' > Val {:2d} loss: {:.3f}'.format((i+1), loss.item()))
    print('accuracy',accuracy)
print(total_acc/cnt2)    

 > Val 81 loss: 0.249
accuracy tensor(0.9233)
 > Val 82 loss: 0.197
accuracy tensor(0.9368)
 > Val 83 loss: 0.535
accuracy tensor(0.8787)
 > Val 84 loss: 0.441
accuracy tensor(0.9019)
 > Val 85 loss: 0.349
accuracy tensor(0.9023)
 > Val 86 loss: 0.435
accuracy tensor(0.8804)
 > Val 87 loss: 0.091
accuracy tensor(0.9648)
 > Val 88 loss: 0.208
accuracy tensor(0.9360)
 > Val 89 loss: 0.439
accuracy tensor(0.8940)
 > Val 90 loss: 0.077
accuracy tensor(0.9673)
 > Val 91 loss: 0.086
accuracy tensor(0.9639)
 > Val 92 loss: 0.207
accuracy tensor(0.9280)
 > Val 93 loss: 0.168
accuracy tensor(0.9380)
 > Val 94 loss: 0.192
accuracy tensor(0.9465)
 > Val 95 loss: 0.103
accuracy tensor(0.9583)
 > Val 96 loss: 0.384
accuracy tensor(0.8977)
 > Val 97 loss: 0.483
accuracy tensor(0.8743)
 > Val 98 loss: 0.233
accuracy tensor(0.9250)
 > Val 99 loss: 0.252
accuracy tensor(0.9180)
 > Val 100 loss: 0.530
accuracy tensor(0.8855)
 > Val 101 loss: 0.276
accuracy tensor(0.9189)
 > Val 102 loss: 0.584
accuracy 

IndexError: ignored

In [0]:
### complete sm_preds_fn
sm_preds = []
for i in range(118):
    
    start_i = i*bs
    end_i = start_i+bs
    xb = temp_ls[start_i:end_i]
    yb = y_f[start_i:end_i]
    target = temp2[start_i:end_i]
        
    out = model(xb)
    sm_preds_val = softmax_mod(out.transpose(1, 2))
    sm_preds_fn = torch.argmax(sm_preds_val, dim=2)+1
    sm_preds.append(sm_preds_fn)

In [0]:
tr = []
pred = []
for i in range(80,108):
    start_i = i*bs
    end_i = start_i+bs
    xb = temp_ls[start_i:end_i]
    yb = y_f[start_i:end_i]
    target = temp2[start_i:end_i]
    s1_true = np.where(target[0] == 3)
    s1_pred = np.where(sm_preds[i][0] == 3)
    tr.append(s1_true)
    pred.append(s1_pred)

In [0]:
for i in range(28):
    print(i)
    print(tr[i])
    print(pred[i])

In [0]:
### 17 epocks ADAM
torch.max(softmax_mod(model(temp[0:64].cuda())[0]).transpose(0,1),1)[1]+1

tensor([4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,
        3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')

In [0]:
### 15 epocks ADAM
torch.max(softmax_mod(model(temp[0:64])[0]).transpose(0,1),1)[1]+1

tensor([4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3,
        3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 4, 4, 4,
        4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2])

In [0]:
## 10 epochs adam
torch.max(softmax_mod(model(temp[0:64])[0]).transpose(0,1),1)[1]+1

tensor([2, 2, 4, 4, 4, 4, 3, 1, 1, 2, 1, 3, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4,
        4, 2, 3, 3, 2, 3, 3, 2, 1, 3, 1, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 2,
        1, 4, 4, 4, 4, 4, 4, 1, 3, 1, 1, 4, 4, 4, 2, 1])

In [0]:
torch.max(y_f[0:64][0].transpose(0,1),1)[1]+1

tensor([4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,
        3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2])

In [0]:
i = 0
bs = 64
start_i = i*bs
end_i = start_i+bs
xb = temp[start_i:end_i]
yb = y_f[start_i:end_i]
target = temp2[start_i:end_i]

In [0]:
yb.shape

torch.Size([64, 4, 64])

In [0]:
def softmax(x): return (x.exp()/(x.exp().sum(-1,keepdim=True)))

In [0]:
out = model(xb)
k1 = softmax(out.transpose(1, 2))

In [0]:
k1.shape

torch.Size([64, 64, 4])

In [0]:
a1 = k1[1][10]
a1.shape

torch.Size([4])

In [0]:
a1

tensor([0.2807, 0.2691, 0.2047, 0.2455], grad_fn=<SelectBackward>)

In [0]:
k23 = torch.argmax(k1, dim=2)

In [0]:
k23

tensor([[1, 0, 0,  ..., 1, 0, 1],
        [3, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 0, 1],
        ...,
        [0, 1, 0,  ..., 1, 1, 1],
        [2, 1, 0,  ..., 1, 0, 1],
        [1, 0, 3,  ..., 1, 0, 1]])

In [0]:
k1[63][0]

tensor([0.2545, 0.2648, 0.2245, 0.2561], grad_fn=<SelectBackward>)

In [0]:
k1[0]

In [0]:
def nll23(soft,target):
  ## soft : 64,64,4
  ## target : 64,64,4 or 64, 64
  
  return -soft[range(target.shape[0]),range(target.shape[1]), target-1]
a12 = nll23(k1, target.long())
a12

tensor([[-0.2521, -0.2443, -0.2466,  ..., -0.2663, -0.2884, -0.2597],
        [-0.2568, -0.2694, -0.2558,  ..., -0.2394, -0.2272, -0.2444],
        [-0.2521, -0.2443, -0.2466,  ..., -0.2394, -0.2272, -0.2444],
        ...,
        [-0.2521, -0.2443, -0.2466,  ..., -0.2762, -0.2614, -0.2754],
        [-0.2568, -0.2242, -0.2108,  ..., -0.2394, -0.2272, -0.2444],
        [-0.2521, -0.2443, -0.2466,  ..., -0.2762, -0.2614, -0.2205]],
       grad_fn=<NegBackward>)

In [0]:
def nll_final(soft,target):
  batch_var = torch.zeros([64, 64] , requires_grad=True)
  out = torch.zeros([1, 1] , requires_grad=True)
  for i in range(64):
    for j in range(64):
      ind = target[i][j]-1
      batch_var[i][j] = soft[i][j][ind]
      out = out + soft[i][j][ind]
  #out = batch_var.mean()
  return -1*(out/(64*64))

In [0]:
a23 = nll_final(k1, target.long())
a23

tensor([[-0.2497]], grad_fn=<MulBackward0>)

In [0]:
knew = [k1[0][0][3], k1[0][1][3], k1[0][2][3]]

In [0]:
knew

[tensor(0.2521, grad_fn=<SelectBackward>),
 tensor(0.2269, grad_fn=<SelectBackward>),
 tensor(0.2368, grad_fn=<SelectBackward>)]

In [0]:
print(k1[1][0][1], k1[1][1][1], k1[1][2][1])

tensor(0.2120, grad_fn=<SelectBackward>) tensor(0.2694, grad_fn=<SelectBackward>) tensor(0.2443, grad_fn=<SelectBackward>)


In [0]:
k1[2][0]

tensor([0.2582, 0.2570, 0.2418, 0.2431], grad_fn=<SelectBackward>)

In [0]:
print(k1[63][0][1], k1[63][1][1], k1[63][2][1])

tensor(0.2648, grad_fn=<SelectBackward>) tensor(0.2571, grad_fn=<SelectBackward>) tensor(0.2335, grad_fn=<SelectBackward>)


In [0]:
a12.shape

torch.Size([64, 64])

In [0]:
target

tensor([[4., 4., 4.,  ..., 2., 2., 2.],
        [2., 2., 2.,  ..., 4., 4., 4.],
        [4., 4., 4.,  ..., 2., 3., 3.],
        ...,
        [4., 4., 4.,  ..., 2., 2., 2.],
        [2., 2., 2.,  ..., 1., 1., 2.],
        [2., 2., 2.,  ..., 1., 1., 1.]])